In [27]:
import copy
from datetime import datetime
from joblib import Parallel, delayed
import lightgbm as lgb
from lightgbm.callback import early_stopping, log_evaluation
import numpy as np
import pandas as pd
import random
from scipy.signal import resample
from sdtw import SoftDTW
from sdtw.distance import SquaredEuclidean

from skopt import gp_minimize
from skopt.space import Real, Integer
from skopt.utils import use_named_args

In [28]:
import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams['figure.figsize'] = [18, 6]

In [29]:
%load_ext memory_profiler

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


In [30]:
from truth import IeeeGroundTruth
from wavelet import apply_wavelet
from peaks import get_peaks_v2
from signal_pross import (
    normalize_signal,
    detrend_w_poly,
    normalize_amplitude_to_1,
    n_moving_avg,
    min_max_scale,
    bandpass,
    get_hr,
    get_hrv
)

In [31]:
class LossFactory:

    def __init__(self, split_size, loss_type = 'mse', gamma = 1.0, mse_weight = None, dtw_weight = None):
        
        if loss_type not in ['mse', 'dtw', 'combined']:
            raise ValueError(f'Loss type [{loss_type}] not supported')
        
        self.split_size = split_size
        self.gamma = gamma
        self.mse_weight = mse_weight
        self.dtw_weight = dtw_weight

        if loss_type == 'mse':
            self.loss_function = self.mse_loss
        elif loss_type == 'dtw':
            self.loss_function = self.soft_dtw_loss
        elif loss_type == 'combined':
            self.loss_function = self.combined_loss
        
    def __call__(self, y_pred, data):
        return self.loss_function(y_pred, data)

    def mse_loss(self, y_pred, data):
        
        y_true = data.get_label()
        num_batches = int(len(y_pred) / self.split_size)
        errs = np.zeros_like(y_true)

        for i in range(num_batches):

            y_true_curr = y_true[i * self.split_size: (i + 1) * self.split_size]
            y_pred_curr = y_pred[i * self.split_size: (i + 1) * self.split_size]
            
            err = y_true_curr - y_pred_curr
            errs[i * self.split_size: (i + 1) * self.split_size] = err

        grad = -2 * errs
        hess = 2 * np.ones_like(y_true)
        return grad, hess

    # def soft_dtw_loss(self, y_pred, data):

    #     y_true = data.get_label()
    #     num_batches = int(len(y_pred) / self.split_size)
    #     grads = np.zeros_like(y_true)
    #     hesses = np.zeros_like(y_true)

    #     for i in range(num_batches):

    #         y_true_curr = y_true[i * self.split_size: (i + 1) * self.split_size]
    #         y_pred_curr = y_pred[i * self.split_size: (i + 1) * self.split_size]
            
    #         grad_curr, hess_curr = self.soft_dtw_loss_helper(y_true_curr, y_pred_curr)
    #         grad_curr = grad_curr.flatten()
    #         hess_curr = hess_curr.flatten()

    #         grads[i * self.split_size: (i + 1) * self.split_size] = grad_curr
    #         hesses[i * self.split_size: (i + 1) * self.split_size] = hess_curr

    #     return grads, hesses
    def soft_dtw_loss(self, y_pred, data):

        def batch_loss_helper(i, y_true, y_pred, split_size):
            
            y_true_curr = y_true[i * split_size: (i + 1) * split_size]
            y_pred_curr = y_pred[i * split_size: (i + 1) * split_size]

            grad_curr, hess_curr = self.soft_dtw_loss_helper(y_true_curr, y_pred_curr)
            grad_curr = grad_curr.flatten()
            hess_curr = hess_curr.flatten()

            return grad_curr, hess_curr

        y_true = data.get_label()
        num_batches = int(len(y_pred) / self.split_size)
        grads = np.zeros_like(y_true)
        hesses = np.zeros_like(y_true)

        results = Parallel(n_jobs = -1)(
            delayed(batch_loss_helper)(
                i, y_true, y_pred, self.split_size
            ) for i in range(num_batches)
        )

        for i, (grad_curr, hess_curr) in enumerate(results):
            grads[i * self.split_size: (i + 1) * self.split_size] = grad_curr
            hesses[i * self.split_size: (i + 1) * self.split_size] = hess_curr

        return grads, hesses
    
    def soft_dtw_loss_helper(self, y_true, y_pred):
        x = y_true.reshape(-1, 1)
        y = y_pred.reshape(-1, 1)
        D = SquaredEuclidean(x, y)
        sdtw = SoftDTW(D, gamma = self.gamma)
        sdtw.compute()
        E = sdtw.grad()
        G = D.jacobian_product(E)
        return G, np.ones(len(G))
    
    def combined_loss(self, y_pred, data):

        if self.mse_weight is None or self.dtw_weight is None:
            raise ValueError('mse_weight and dtw_weight must be set before calling combined_loss')

        mse_grads, mse_hesses = self.mse_loss(y_pred, data)
        dtw_grads, dtw_hesses = self.soft_dtw_loss(y_pred, data)

        combined_grad = self.mse_weight * mse_grads + self.dtw_weight * dtw_grads
        combined_hess = self.mse_weight * mse_hesses + self.dtw_weight * dtw_hesses

        return combined_grad, combined_hess

In [35]:
class LonePineGBM:
    
    def __init__(self, truths, label_col = 'bvp', subject_col = 'subject',
                # model customization
                model_type = 'gbdt', random_state = None, loss_type = 'mse', excluded_subject = None,
                # hyperparameters
                n_estimators = 100, split_size = 1280, learning_rate = 0.1, test_size = 0.3, early_stopping_rounds = 50,
                mse_weight = None, dtw_weight = None, data_beg = 1000, data_end = 10000, batches = 1, finetune = True,
                min_bandpass_freq = 0.67, max_bandpass_freq = 3.0, bandpass_order = 4,
                predicted_peaks_prominence = 0.22, true_peaks_prominence = 0.15,
                # hyperparams from LightGBM docs
                max_depth = 7, num_leaves = 75, max_bin = 255,
                num_feats_per_channel = 3, skip_amount = 15):
        
        if model_type not in ['gbdt', 'rf']:
            raise ValueError(f'Model type [{model_type}] not supported')
        
        self.label_col = label_col
        self.subject_col = subject_col

        self.model_type = model_type
        self.random_state = random_state
        self.excluded_subject = excluded_subject

        self.n_estimators = n_estimators
        self.split_size = split_size
        self.learning_rate = learning_rate
        self.test_size = test_size
        self.early_stopping_rounds = early_stopping_rounds

        self.data_beg = data_beg
        self.data_end = data_end
        self.finetune = finetune

        self.min_bandpass_freq = min_bandpass_freq
        self.max_bandpass_freq = max_bandpass_freq
        self.bandpass_order = bandpass_order
        self.predicted_peaks_prominence = predicted_peaks_prominence
        self.true_peaks_prominence = true_peaks_prominence

        self.max_depth = max_depth
        self.num_leaves = num_leaves
        self.max_bin = max_bin

        self.num_feats_per_channel = num_feats_per_channel
        self.skip_amount = skip_amount

        self.gbm = None
        self.training_loss = None
        self.test_loss = None

        self.given_data = self.prepare_dataset_from_subjects(truths, data_beg = data_beg, data_end = data_end)
        self.features = list(self.given_data.drop(columns = [self.label_col, self.subject_col]).columns)
        if self.excluded_subject is not None:
            self.given_data = self.given_data[self.given_data[self.subject_col] != self.excluded_subject]

        if self.random_state is not None:
            random.seed(self.random_state)
        splits = self.split_data()
        self.train_split_indices = random.sample(range(len(splits)), int(len(splits) * (1 - self.test_size)))
        self.train_splits = [splits[i] for i in self.train_split_indices]
        self.test_splits = [splits[i] for i in range(len(splits)) if i not in self.train_split_indices]
        
        self.train_data = []
        batch_size = len(self.train_splits) // batches
        print(f'Rows per batch: {batch_size * self.split_size}')
        for batch_num in range(batches):
            batch_split_idxs = random.sample(range(len(self.train_splits)), batch_size)
            batch_splits = [self.train_splits[i] for i in batch_split_idxs]
            self.train_splits = [self.train_splits[i] for i in range(len(self.train_splits)) if i not in batch_split_idxs]

            train_indices = [idx for split in batch_splits for idx in split]
            training_rows = self.given_data.iloc[train_indices].drop(columns = [self.subject_col])
            train_X = training_rows.drop(columns = [self.label_col]).to_numpy()
            train_y = training_rows[self.label_col].to_numpy()

            batch_data = lgb.Dataset(train_X, train_y, free_raw_data = False)
            self.train_data.append(batch_data)

        test_indices = [idx for split in self.test_splits for idx in split]
        test_rows = self.given_data.iloc[test_indices].drop(columns = [self.subject_col])
        test_X = test_rows.drop(columns = [self.label_col]).to_numpy()
        test_y = test_rows[self.label_col].to_numpy()
        self.test_data = lgb.Dataset(test_X, test_y, free_raw_data = False)

        self.loss = LossFactory(self.split_size, loss_type = loss_type, mse_weight = mse_weight, dtw_weight = dtw_weight)
    
    def split_data(self, to_exclude = None):
        
        data_in_use = self.given_data if to_exclude is None else self.given_data[~self.given_data[self.subject_col].isin(to_exclude)]

        subject_indices = data_in_use.groupby(self.subject_col).indices
        splits = []
        for _, indices in subject_indices.items():
            
            n_splits = len(indices) // self.split_size
            if n_splits > 0:

                subject_splits = []
                for i in range(n_splits):
                    split_start = i * self.split_size
                    split_end = (i + 1) * self.split_size
                    subject_split = indices[split_start: split_end]
                    subject_splits.append(subject_split)
                
                splits.extend(subject_splits)
        
        return splits

    def fit(self):
        t1 = datetime.today()
        
        params = {
            'metric': 'None',
            'verbosity': -1,
            'learning_rate': self.learning_rate,
            'objective': 'regression',
            'boosting': self.model_type,
            'max_depth': self.max_depth,
            'num_leaves': self.num_leaves,
            'max_bin': self.max_bin,
        }
    
        if self.model_type == 'rf':
            params['bagging_freq'] = 1
            params['bagging_fraction'] = 0.8


        training_loss_key = 'hr_err'
        feval = self.hr_error_eval_metric
        
        training_meta = {}

        for train_data in self.train_data:
            
            if self.model_type == 'gbdt':
                self.gbm = lgb.train(
                    params,
                    train_data,
                    valid_sets = [train_data, self.test_data],
                    valid_names=['train', 'test'],
                    fobj = self.loss,
                    num_boost_round = self.n_estimators,
                    feval=feval,
                    callbacks=[
                        early_stopping(stopping_rounds = self.early_stopping_rounds),
                        log_evaluation(period=5)
                    ],
                    evals_result = training_meta,
                    init_model = self.gbm
                )
            else:
                self.gbm = lgb.train(
                    params,
                    train_data,
                    valid_sets = [train_data, self.test_data],
                    valid_names=['train', 'test'],
                    num_boost_round = self.n_estimators,
                    feval=feval,
                    callbacks=[
                        early_stopping(stopping_rounds = self.early_stopping_rounds),
                        log_evaluation(period=5)
                    ],
                    evals_result = training_meta,
                )

            mse, hr_err, hr_err_sq = self.eval()
            print(f'Before fine-tuning: MSE = {mse}, HR error = {hr_err}, HR error (squared) = {hr_err_sq}')

            if self.model_type == 'gbdt' and self.finetune:
                
                print('\n\nFine-tuning...')
                gbm_copy = copy.deepcopy(self.gbm)
                pred = gbm_copy.predict(train_data.get_data())
                
                # new_targ = train_data.get_label() - pred
                new_targ = np.ones(len(pred))
                nsplits = len(pred) // self.split_size
                labels = train_data.get_label()
                for i in range(nsplits):
                    pred_curr = pred[i * self.split_size: (i + 1) * self.split_size]
                    label_curr = labels[i * self.split_size: (i + 1) * self.split_size]
                    hr_err = self.get_hr_error(pred_curr, label_curr, square = True)
                    new_targ[i * self.split_size: (i + 1) * self.split_size] = hr_err
                
                new_train_data = lgb.Dataset(train_data.get_data(), label = new_targ)

                self.gbm = lgb.train(
                    params,
                    new_train_data,
                    valid_sets = [new_train_data, self.test_data],
                    valid_names=['train', 'test'],
                    fobj = self.loss,
                    num_boost_round = self.n_estimators // 2,
                    feval=feval,
                    callbacks=[
                        early_stopping(stopping_rounds = self.early_stopping_rounds // 2),
                        log_evaluation(period=5)
                    ],
                    evals_result = training_meta,
                    init_model = gbm_copy
                )

            

        self.training_loss = training_meta['train'][training_loss_key]
        self.test_loss = training_meta['test'][training_loss_key]
        print(f'Finished training in {datetime.today() - t1}')

    def predict(self, X):
        return self.gbm.predict(X)

    def eval(self):
        
        test_X = self.test_data.get_data()
        test_y = self.test_data.get_label()
        nsplits = int(len(test_X) / self.split_size)
        errs = []
        mses = np.zeros(len(test_X))
        
        for i in range(nsplits):

            curr_pred = self.predict(test_X[i * self.split_size: (i + 1) * self.split_size, :])
            curr_true = test_y[i * self.split_size: (i + 1) * self.split_size]
            curr_true, curr_pred = self.process_signal(curr_true, curr_pred, smoothing_window = 5, use_bandpass = True)
            
            mses[i * self.split_size: (i + 1) * self.split_size] = curr_true - curr_pred
            hr_err = self.get_hr_error(curr_true, curr_pred, square = False)
            errs.append(hr_err)
        
        return np.mean(np.square(mses)), np.mean(errs), np.mean(np.square(errs))
    
    def validate(self):

        test_X = self.test_data.get_data()
        test_y = self.test_data.get_label()
        nsplits = int(len(test_X) / self.split_size)
        
        errors = []
        for i in range(nsplits):

            curr_pred = self.predict(test_X[i * self.split_size: (i + 1) * self.split_size, :])
            curr_true = test_y[i * self.split_size: (i + 1) * self.split_size]
            curr_true, curr_pred = self.process_signal(curr_true, curr_pred, smoothing_window = 5, use_bandpass = True)
            
            mse = np.mean(np.square(curr_true - curr_pred))
            hr_err = self.get_hr_error(curr_true, curr_pred, square = False)
            hrv_err = self.get_hrv_error(curr_true, curr_pred, square = False)
            peaks_err = self.get_peaks_error(curr_true, curr_pred, square = False)
            errors.append({
                'mse': mse,
                'hr_err': hr_err,
                'hrv_err': hrv_err,
                'peaks_err': peaks_err
            })

        return errors

    def plot_loss(self):
        if self.training_loss is not None and self.test_loss is not None:
            training_loss_normed = min_max_scale(self.training_loss)
            test_loss_normed = min_max_scale(self.test_loss)
            plt.plot(training_loss_normed, label = 'training loss')
            plt.plot(test_loss_normed, label = 'test loss')
            plt.legend()
        
    def get_model_stats(self):

        model_info = self.gbm.dump_model()
        tree_depths = []

        for tree_info in model_info['tree_info']:
            tree_structure = tree_info['tree_structure']
            
            # Recursive function to compute the depth of a tree
            def calculate_depth(node, current_depth=0):
                if 'leaf_value' in node:
                    return current_depth
                else:
                    left_depth = calculate_depth(node['left_child'], current_depth + 1)
                    right_depth = calculate_depth(node['right_child'], current_depth + 1)
                    return max(left_depth, right_depth)

            tree_depth = calculate_depth(tree_structure)
            tree_depths.append(tree_depth)
        

        print(f'Best test loss: {min(self.test_loss)}\n')
        print('Tree depth stats:')
        print('Min tree depth:', min(tree_depths))
        print('Max tree depth:', max(tree_depths))
        print('Avg tree depth:', np.mean(tree_depths))
        print('\nFeature importances:')
        display(self.get_feature_importances())
    
    def get_feature_importances(self):
        importances = self.gbm.feature_importance(importance_type='gain')
        feature_importances = pd.DataFrame({'feature': self.features, 'importance': importances})
        feature_importances = feature_importances.sort_values('importance', ascending=False)
        return feature_importances
    
    def hr_error_eval_metric(self, y_pred, eval_data):
        y_true = eval_data.get_label()
        nsplits = int(len(y_pred) / self.split_size)
        hr_err = []
        for i in range(nsplits):
            curr_pred = y_pred[i * self.split_size: (i + 1) * self.split_size]
            curr_true = y_true[i * self.split_size: (i + 1) * self.split_size]
            curr_true, curr_pred = self.process_signal(curr_true, curr_pred, smoothing_window = 10, use_bandpass = True)
            hr_err.append(self.get_hr_error(curr_true, curr_pred, square = False))
        return 'hr_err', np.mean(hr_err), False
    
    def get_hr_error(self, y_true, y_pred, square = True):

        true_peaks, _ = self.get_true_peaks(y_true)
        pred_peaks, _ = self.get_predicted_peaks(y_pred)

        if len(true_peaks) >= 2:
            true_ibis = np.diff(true_peaks) / 64
            true_hr = 60 / np.mean(true_ibis)
        else:
            true_hr = 0

        if len(pred_peaks) >= 2:
            pred_ibis = np.diff(pred_peaks) / 64
            pred_hr = 60 / np.mean(pred_ibis)
        else:
            pred_hr = 0
        
        if square:
            return np.power(true_hr - pred_hr, 2)
        return abs(true_hr - pred_hr)
    
    def get_peaks_error(self, y_true, y_pred, square = True):
        true_peaks, _ = self.get_true_peaks(y_true)
        pred_peaks, _ = self.get_predicted_peaks(y_pred)
        if square:
            return np.power(len(true_peaks) - len(pred_peaks), 2)
        return abs(len(true_peaks) - len(pred_peaks))
    
    def get_hrv_error(self, y_true, y_pred, square = True):

        true_peaks, _ = self.get_true_peaks(y_true)
        pred_peaks, _ = self.get_predicted_peaks(y_pred)

        if len(true_peaks) >= 2:
            true_ibis = np.diff(true_peaks) / 64
            true_hrv = get_hrv(true_ibis)
        else:
            true_hrv = 0

        if len(pred_peaks) >= 2:
            pred_ibis = np.diff(pred_peaks) / 64
            pred_hrv = get_hrv(pred_ibis)
        else:
            pred_hrv = 0
        
        if square:
            return np.power(true_hrv - pred_hrv, 2)
        return abs(true_hrv - pred_hrv)
    
    def process_signal(self, y_true, y_pred, smoothing_window = 10, use_bandpass = False):
    
        orig_len = len(y_pred)
        y_pred = n_moving_avg(y_pred, smoothing_window)
        y_pred = resample(y_pred, orig_len)
        if use_bandpass:
            y_pred = bandpass(y_pred, 64, [self.min_bandpass_freq, self.max_bandpass_freq], self.bandpass_order)
        y_pred = min_max_scale(y_pred)
        
        y_true = n_moving_avg(y_true, 20)
        y_true = resample(y_true, orig_len)
        if use_bandpass:
            y_true = bandpass(y_true, 64, [self.min_bandpass_freq, self.max_bandpass_freq], self.bandpass_order)
        y_true = min_max_scale(y_true)
        
        return y_true, y_pred
    
    def get_predicted_peaks(self, signal):
        return get_peaks_v2(signal, 64, 3.0, -1, prominence = self.predicted_peaks_prominence, with_min_dist = True, with_valleys = False)
    def get_true_peaks(self, signal):
        return get_peaks_v2(signal, 64, 3.0, -1, prominence = self.true_peaks_prominence, with_min_dist = True, with_valleys = False)

    def prepare_dataset_from_subjects(self, truths, data_beg = 1000, data_end = 2000):
        data_arr = []
        for i in range(len(truths)):    
            truth = truths[i]
            data = truth.prepare_data_for_ml(self.num_feats_per_channel, self.skip_amount)
            data = data.iloc[data_beg: data_end, :]
            data['subject'] = i + 1
            data_arr.append(data)
        return pd.concat(data_arr)


In [33]:
def subjectwise_kfold(truths, model_type = 'gbdt', random_state = None, loss_type = 'mse',
                    n_estimators = 100, split_size = 1280, learning_rate = 0.1, test_size = 0.3, early_stopping_rounds = 50,
                    mse_weight = None, dtw_weight = None, data_beg = 1000, data_end = 10000, batches = 1, finetune = True, 
                    min_bandpass_freq = 0.67, max_bandpass_freq = 3.0, bandpass_order = 4,
                    predicted_peaks_prominence = 0.22, true_peaks_prominence = 0.15,
                    max_depth = 7, num_leaves = 75, max_bin = 255, num_feats_per_channel = 3, skip_amount = 15,
                    rounds_per_model = 1, collect = False):
        
        models = {}
        for subj_idx in range(len(truths)):
            models[subj_idx + 1] = []
            for i in range(rounds_per_model):
                
                print(f'\n\nTraining excluding subject {subj_idx + 1}...\n')
                mod = LonePineGBM(
                    truths = truths,
                    model_type = model_type,
                    random_state = random_state,
                    loss_type = loss_type,
                    n_estimators = n_estimators,
                    split_size = split_size,
                    learning_rate = learning_rate,
                    test_size = test_size,
                    early_stopping_rounds = early_stopping_rounds,
                    mse_weight = mse_weight,
                    dtw_weight = dtw_weight,
                    data_beg = data_beg,
                    data_end = data_end,
                    batches = batches,
                    finetune = finetune,
                    min_bandpass_freq = min_bandpass_freq,
                    max_bandpass_freq = max_bandpass_freq,
                    bandpass_order = bandpass_order,
                    predicted_peaks_prominence = predicted_peaks_prominence,
                    true_peaks_prominence = true_peaks_prominence,
                    max_depth = max_depth,
                    num_leaves = num_leaves,
                    max_bin = max_bin,
                    num_feats_per_channel = num_feats_per_channel,
                    skip_amount = skip_amount,
                    excluded_subject = subj_idx + 1
                )
                mod.fit()
                models[subj_idx + 1].append(mod)
        
        model_performances = {}
        for subj_idx in models:
            model_performances[subj_idx] = []
            for i in range(rounds_per_model):
                mod = models[subj_idx][i]
                
                if collect:
                    model_performances[subj_idx].append(mod.validate())
                else:
                    model_performances[subj_idx].append(mod.eval())

                
        if collect:
            return model_performances

        mean_hr_score = np.mean([model_performances[subj_idx][i][1] for subj_idx in model_performances for i in range(rounds_per_model)])
        return mean_hr_score, models, model_performances
    

class LonePineOptimizer:

    def __init__(self, truths):
        self.truths = truths
    
    def objective(self, n_estimators = 100, split_size = 1280, learning_rate = 0.1, test_size = 0.3, early_stopping_rounds = 50,
                    mse_weight = None, dtw_weight = None, data_beg = 1000, data_end = 2000, batches = 1, finetune = True, 
                    min_bandpass_freq = 0.67, max_bandpass_freq = 3.0, bandpass_order = 4,
                    predicted_peaks_prominence = 0.22, true_peaks_prominence = 0.15,
                    max_depth = 7, num_leaves = 75, max_bin = 255, num_feats_per_channel = 3, skip_amount = 15):

        hr_score, _, _ = subjectwise_kfold(
            self.truths,
            model_type = 'gbdt',
            random_state = None,
            loss_type = 'combined',
            n_estimators = n_estimators,
            split_size = split_size,
            learning_rate = learning_rate,
            early_stopping_rounds = 50,
            mse_weight = mse_weight,
            dtw_weight = dtw_weight,
            data_beg = data_beg,
            data_end = data_end,
            batches = batches,
            finetune = finetune,
            min_bandpass_freq = min_bandpass_freq,
            max_bandpass_freq = max_bandpass_freq,
            bandpass_order = bandpass_order,
            predicted_peaks_prominence = predicted_peaks_prominence,
            true_peaks_prominence = true_peaks_prominence,
            max_depth = max_depth,
            num_leaves = num_leaves,
            max_bin = max_bin,
            num_feats_per_channel = num_feats_per_channel,
            skip_amount = skip_amount,
        )
        return hr_score
    
    def optimize(self, n_calls = 50):

        space = [
            Integer(50, 300, name = "n_estimators"),
            Integer(640, 1280, name = "split_size"),
            Real(0.002, 0.5, name = "learning_rate"),
            Integer(10, 100, name = "early_stopping_rounds"),
            Real(0.0, 1.0, name = "mse_weight"),
            Real(0.0, 1.0, name = "dtw_weight"),
            Integer(1000, 4000, name = "data_beg"),
            Integer(6000, 10000, name = "data_end"),
            Integer(1, 8, name = "batches"),
            Real(0.4, 1.0, name = "min_bandpass_freq"),
            Real(2.5, 4.0, name = "max_bandpass_freq"),
            Integer(2, 6, name = "bandpass_order"),
            Real(0.1, 0.75, name = "predicted_peaks_prominence"),
            Real(0.1, 0.5, name = "true_peaks_prominence"),
            Integer(3, 10, name = "max_depth"),
            Integer(30, 140, name = "num_leaves"),
            Integer(100, 300, name = "max_bin"),
            Integer(3, 10, name = "num_feats_per_channel"),
            Integer(5, 25, name = "skip_amount"),
        ]

        @use_named_args(space)
        def wrapped_objective(**params):
            return self.objective(**params)
        
        result = gp_minimize(
            wrapped_objective, space, n_calls=n_calls, random_state=42, verbose=1
        )

        return result

In [ ]:
# optimizer = LonePineOptimizer(truths)
# result = optimizer.optimize(n_calls = 50)

In [ ]:
# space = [
#             Integer(50, 300, name = "n_estimators"),
#             Integer(640, 1280, name = "split_size"),
#             Real(0.002, 0.5, name = "learning_rate"),
#             Integer(10, 100, name = "early_stopping_rounds"),
#             Real(0.0, 1.0, name = "mse_weight"),
#             Real(0.0, 1.0, name = "dtw_weight"),
#             Integer(1000, 4000, name = "data_beg"),
#             Integer(6000, 10000, name = "data_end"),
#             Integer(1, 8, name = "batches"),
#             Real(0.4, 1.0, name = "min_bandpass_freq"),
#             Real(2.5, 4.0, name = "max_bandpass_freq"),
#             Integer(2, 6, name = "bandpass_order"),
#             Real(0.1, 0.75, name = "predicted_peaks_prominence"),
#             Real(0.1, 0.5, name = "true_peaks_prominence"),
#             Integer(3, 10, name = "max_depth"),
#             Integer(30, 140, name = "num_leaves"),
#             Integer(100, 300, name = "max_bin"),
#             Integer(3, 10, name = "num_feats_per_channel"),
#             Integer(5, 25, name = "skip_amount"),
#         ]

optimization_res = [188,
 993,
 0.22844583483861589,
 16,
 0.913467044583398,
 1.0,
 3909,
 7474,
 5,
 0.9421772128909808,
 3.6351090994830813,
 4,
 0.17087564911262462,
 0.322741927274642,
 6,
 34,
 235,
 8,
 12]

# result.x

In [ ]:
# np.mean([model_performances[sub][i][1] for sub in model_performances for i in range(3)])

In [40]:
options = [
    {
        'minmax': False,
        'use_wavelet': True,
        'use_bandpass': False
    }
]
rpm = 10

scores = []

for option in options:
    
    truths = []
    for subject in range(1, 8):

        truth = IeeeGroundTruth(subject, 1, directory = 'channel_data3')
        truth.align_rgb_bvp()
        truth.fill_nans()
        truth.process_rgb(
            minmax = option['minmax'],
            use_wavelet = option['use_wavelet'],
            use_bandpass = option['use_bandpass']
        )
        truth.process_bvp()
        truths.append(truth)


    res = subjectwise_kfold(
        truths, model_type = 'gbdt', random_state = None, loss_type = 'combined', rounds_per_model = rpm,
        n_estimators = 188, split_size = 960, learning_rate = 0.01,
        early_stopping_rounds = 16, mse_weight = 0.2, dtw_weight = 0.8, data_beg = 8000, data_end = 10180,
        batches = 5, min_bandpass_freq = 0.7, max_bandpass_freq = 4.0, bandpass_order = 4,
        predicted_peaks_prominence = 0.17087564911262462, true_peaks_prominence = 0.322741927274642, max_depth = 6,
        num_leaves = 34, max_bin = 235, num_feats_per_channel = 8, skip_amount = 12, collect = True
    )

    scores.append(res)



Training excluding subject 1...

Rows per batch: 960
Training until validation scores don't improve for 16 rounds


/Users/samuelhmorton/opt/anaconda3/lib/python3.7/site-packages/lightgbm/engine.py:260: UserWarning: 'evals_result' argument is deprecated and will be removed in a future release of LightGBM. Pass 'record_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'evals_result' argument is deprecated and will be removed in a future release of LightGBM. "


[5]	train's hr_err: 9.17698	test's hr_err: 11.2323
[10]	train's hr_err: 9.17698	test's hr_err: 9.98042
[15]	train's hr_err: 9.07068	test's hr_err: 15.4588
[20]	train's hr_err: 9.17698	test's hr_err: 18.2739
[25]	train's hr_err: 9.17698	test's hr_err: 22.5476
Early stopping, best iteration is:
[10]	train's hr_err: 9.17698	test's hr_err: 9.98042
Before fine-tuning: MSE = 0.12074863753011056, HR error = 15.03102368100132, HR error (squared) = 445.73479807468544


Fine-tuning...
Training until validation scores don't improve for 8 rounds
[15]	train's hr_err: 96.8421	test's hr_err: 11.3439
[20]	train's hr_err: 96.9484	test's hr_err: 9.98042
Early stopping, best iteration is:
[14]	train's hr_err: 96.8421	test's hr_err: 9.88508
[15]	train's hr_err: 35.6897	test's hr_err: 9.94871
Training until validation scores don't improve for 16 rounds
[20]	train's hr_err: 31.3181	test's hr_err: 10.0124
[25]	train's hr_err: 35.4992	test's hr_err: 10.0683
[30]	train's hr_err: 39.6709	test's hr_err: 11.0729


/Users/samuelhmorton/opt/anaconda3/lib/python3.7/site-packages/lightgbm/engine.py:260: UserWarning: 'evals_result' argument is deprecated and will be removed in a future release of LightGBM. Pass 'record_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'evals_result' argument is deprecated and will be removed in a future release of LightGBM. "


Training until validation scores don't improve for 16 rounds
[5]	train's hr_err: 7.94405	test's hr_err: 29.7567
[10]	train's hr_err: 5.59816	test's hr_err: 32.2083
[15]	train's hr_err: 5.51551	test's hr_err: 32.4083
Early stopping, best iteration is:
[3]	train's hr_err: 6.16848	test's hr_err: 25.8475
Before fine-tuning: MSE = 0.08514426620984028, HR error = 23.19875374891734, HR error (squared) = 1010.1515841543537


Fine-tuning...
Training until validation scores don't improve for 8 rounds
[5]	train's hr_err: 67.2701	test's hr_err: 23.3188
[10]	train's hr_err: 67.2701	test's hr_err: 25.7861
[15]	train's hr_err: 67.2701	test's hr_err: 23.1587
[20]	train's hr_err: 67.2701	test's hr_err: 23.3443
Early stopping, best iteration is:
[15]	train's hr_err: 67.2701	test's hr_err: 23.1587
Training until validation scores don't improve for 16 rounds
[20]	train's hr_err: 6.97965	test's hr_err: 10.3625
[25]	train's hr_err: 2.91474	test's hr_err: 15.9889
[30]	train's hr_err: 2.91474	test's hr_err: 1

...

Training until validation scores don't improve for 16 rounds
[5]	train's hr_err: 0.304762	test's hr_err: 32.0814
[10]	train's hr_err: 0.228317	test's hr_err: 22.5317
[15]	train's hr_err: 0.152042	test's hr_err: 27.0376
[20]	train's hr_err: 0.152042	test's hr_err: 26.7869
[25]	train's hr_err: 1.00053	test's hr_err: 25.0342
Early stopping, best iteration is:
[11]	train's hr_err: 0.228317	test's hr_err: 11.7819
Before fine-tuning: MSE = 0.10725229682470455, HR error = 13.558883797830022, HR error (squared) = 279.17830258151736


Fine-tuning...
Training until validation scores don't improve for 8 rounds
[15]	train's hr_err: 68.495	test's hr_err: 12.0473
[20]	train's hr_err: 68.495	test's hr_err: 12.0253
Early stopping, best iteration is:
[12]	train's hr_err: 68.495	test's hr_err: 11.7694
Training until validation scores don't improve for 16 rounds
[15]	train's hr_err: 0.0921733	test's hr_err: 26.0263
[20]	train's hr_err: 5.59139	test's hr_err: 29.0137
[25]	train's hr_err: 5.51196	test's hr

/Users/samuelhmorton/opt/anaconda3/lib/python3.7/site-packages/lightgbm/engine.py:260: UserWarning: 'evals_result' argument is deprecated and will be removed in a future release of LightGBM. Pass 'record_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'evals_result' argument is deprecated and will be removed in a future release of LightGBM. "


Training until validation scores don't improve for 16 rounds
[5]	train's hr_err: 20.026	test's hr_err: 2.80674
[10]	train's hr_err: 19.2849	test's hr_err: 4.9959
[15]	train's hr_err: 19.5591	test's hr_err: 6.84336
[20]	train's hr_err: 19.4675	test's hr_err: 8.56528
Early stopping, best iteration is:
[4]	train's hr_err: 20.026	test's hr_err: 2.78352
Before fine-tuning: MSE = 0.0868138939633783, HR error = 5.176063150058833, HR error (squared) = 28.164525546819032


Fine-tuning...
[5]	train's hr_err: 79.9075	test's hr_err: 2.89445
Training until validation scores don't improve for 8 rounds
[10]	train's hr_err: 78.4563	test's hr_err: 3.58039
Early stopping, best iteration is:
[5]	train's hr_err: 79.9075	test's hr_err: 2.89445
Training until validation scores don't improve for 16 rounds
[10]	train's hr_err: 1.14833	test's hr_err: 9.81193
[15]	train's hr_err: 1.14833	test's hr_err: 8.1552
[20]	train's hr_err: 1.14833	test's hr_err: 9.71936
Early stopping, best iteration is:
[6]	train's hr_e

/Users/samuelhmorton/opt/anaconda3/lib/python3.7/site-packages/lightgbm/engine.py:260: UserWarning: 'evals_result' argument is deprecated and will be removed in a future release of LightGBM. Pass 'record_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'evals_result' argument is deprecated and will be removed in a future release of LightGBM. "


Training until validation scores don't improve for 16 rounds
[5]	train's hr_err: 0.121133	test's hr_err: 28.5298
[10]	train's hr_err: 0.371664	test's hr_err: 28.3355
[15]	train's hr_err: 0.189386	test's hr_err: 44.1989
Early stopping, best iteration is:
[1]	train's hr_err: 0.0527261	test's hr_err: 28.2232
Before fine-tuning: MSE = 0.10758356659073419, HR error = 26.566960501648442, HR error (squared) = 828.2815547837921


Fine-tuning...
Training until validation scores don't improve for 8 rounds
[5]	train's hr_err: 60.7458	test's hr_err: 26.6184
[10]	train's hr_err: 61.2515	test's hr_err: 26.4294
[15]	train's hr_err: 60.7458	test's hr_err: 27.9542
Early stopping, best iteration is:
[11]	train's hr_err: 60.7458	test's hr_err: 26.4011
Training until validation scores don't improve for 16 rounds
[15]	train's hr_err: 1.3932	test's hr_err: 50.1481
[20]	train's hr_err: 0.535125	test's hr_err: 51.193
[25]	train's hr_err: 0.535125	test's hr_err: 49.8418
Early stopping, best iteration is:
[12]	

In [47]:
# min_bandpass_freq = 0.9421772128909808
# max_bandpass_freq = 3.6351090994830813
scores[0][1]

[[{'mse': 0.1020667863153025,
   'hr_err': 29.580685754720292,
   'hrv_err': 0.2698757404091039,
   'peaks_err': 8},
  {'mse': 0.13692738324253304,
   'hr_err': 2.5461778914464333,
   'hrv_err': 0.1826841603789613,
   'peaks_err': 0},
  {'mse': 0.13030660706137265,
   'hr_err': 7.403497195644988,
   'hrv_err': 0.306363422305563,
   'peaks_err': 1}],
 [{'mse': 0.07525562690908212,
   'hr_err': 9.46144538168808,
   'hrv_err': 0.5756507796426258,
   'peaks_err': 2},
  {'mse': 0.11745277574381617,
   'hr_err': 13.001783611639667,
   'hrv_err': 0.24677190580137132,
   'peaks_err': 5},
  {'mse': 0.09966862598322009,
   'hr_err': 6.10517212705804,
   'hrv_err': 0.3760437656535905,
   'peaks_err': 1}],
 [{'mse': 0.11997090003190579,
   'hr_err': 26.816242389376285,
   'hrv_err': 0.456709760261418,
   'peaks_err': 8},
  {'mse': 0.13250884687977388,
   'hr_err': 6.546431265292597,
   'hrv_err': 0.0766223466787661,
   'peaks_err': 2},
  {'mse': 0.11981418306477892,
   'hr_err': 25.320658722720573

In [54]:
res = scores[0]
for sub in res:
    print(f'Subject {sub}:')
    sub_errs = {'hr': [], 'hrv': [], 'peaks': []}
    for subres in res[sub]:
        hr = round(np.mean([d['hr_err'] for d in subres]), 2)
        hrv = round(np.mean([d['hrv_err'] for d in subres]), 2)
        peaks = round(np.mean([d['peaks_err'] for d in subres]), 2)
        sub_errs['hr'].append(hr)
        sub_errs['hrv'].append(hrv)
        sub_errs['peaks'].append(peaks)

        print(f'\t- HR: {hr}; HRV: {hrv}; Peaks: {peaks}')
    
    print(f'\tMean HR: {round(np.mean(sub_errs["hr"]), 2)}; Mean HRV: {round(np.mean(sub_errs["hrv"]), 2)}; Mean Peaks: {round(np.mean(sub_errs["peaks"]), 2)}')
    print()

Subject 1:
	- HR: 13.18; HRV: 0.25; Peaks: 3.0
	- HR: 9.52; HRV: 0.4; Peaks: 2.67
	- HR: 19.56; HRV: 0.23; Peaks: 6.0
	- HR: 12.59; HRV: 0.23; Peaks: 3.67
	- HR: 8.68; HRV: 0.47; Peaks: 4.33
	- HR: 7.22; HRV: 0.57; Peaks: 3.0
	- HR: 15.36; HRV: 0.42; Peaks: 5.67
	- HR: 18.05; HRV: 0.27; Peaks: 5.0
	- HR: 22.97; HRV: 0.38; Peaks: 6.67
	- HR: 17.53; HRV: 0.15; Peaks: 5.0
	Mean HR: 14.47; Mean HRV: 0.34; Mean Peaks: 4.5

Subject 2:
	- HR: 14.62; HRV: 0.34; Peaks: 2.33
	- HR: 11.73; HRV: 0.87; Peaks: 2.67
	- HR: 3.57; HRV: 0.33; Peaks: 3.0
	- HR: 21.61; HRV: 0.33; Peaks: 6.33
	- HR: 17.46; HRV: 0.46; Peaks: 4.67
	- HR: 12.23; HRV: 0.45; Peaks: 4.0
	- HR: 16.51; HRV: 0.19; Peaks: 4.67
	- HR: 32.38; HRV: 0.17; Peaks: 9.0
	- HR: 10.73; HRV: 0.32; Peaks: 4.0
	- HR: 12.25; HRV: 0.25; Peaks: 2.0
	Mean HR: 15.31; Mean HRV: 0.37; Mean Peaks: 4.27

Subject 3:
	- HR: 12.72; HRV: 0.33; Peaks: 4.0
	- HR: 18.54; HRV: 0.49; Peaks: 8.0
	- HR: 5.69; HRV: 1.06; Peaks: 1.33
	- HR: 1.46; HRV: 0.59; Peaks: 0.

In [ ]:
# test_subject = 7
# test_subject_truth = truths[test_subject - 1]

# mod = LonePineGBM(
#     truths, n_estimators = 188, loss_type = 'combined', split_size = 960, learning_rate = 0.05,#learning_rate = 0.22844583483861589,
#     early_stopping_rounds = 16, mse_weight = 0.1, dtw_weight = 0.9, data_beg = 4000, data_end = 7500,
#     batches = 5, min_bandpass_freq = 0.9421772128909808, max_bandpass_freq = 3.6351090994830813, bandpass_order = 4,
#     predicted_peaks_prominence = 0.17087564911262462, true_peaks_prominence = 0.322741927274642, max_depth = 6,
#     num_leaves = 34, max_bin = 235, num_feats_per_channel = 8, skip_amount = 12,
#     excluded_subject = test_subject, model_type = 'gbdt', finetune = True
# )
# mod.fit()

In [ ]:
mse, hr_err, hr_err_sq = mod.eval()

print(f'\n\nMSE: {mse}')
print(f'HR error: {hr_err}')
print(f'HR error squared: {hr_err_sq}\n\n')

mod.get_model_stats()
mod.plot_loss()

In [ ]:
beg = 2000
end = beg + 640

model = mod

data = truths[2].prepare_data_for_ml(8, 12)
x = data.drop(columns = ['bvp']).to_numpy()
y = data['bvp'].to_numpy()

def measure_code_block():
    targ = y[beg: end]
    pred = model.predict(x)
    pred = pred[beg: end]
    targ, pred = mod.process_signal(targ, pred, use_bandpass=True)
    return targ, pred

%memit targ, pred = measure_code_block()

pred_peaks, _ = mod.get_predicted_peaks(pred)
true_peaks, _ = mod.get_true_peaks(targ)

plt.plot(targ)
plt.plot(pred)
plt.scatter(pred_peaks, pred[pred_peaks], c='r')
plt.scatter(true_peaks, targ[true_peaks], c='g')

pred_ibis = np.diff(pred_peaks) / 64
true_ibis = np.diff(true_peaks) / 64
pred_hr = get_hr(pred_ibis)
true_hr = get_hr(true_ibis)
print(f'True HR: {true_hr}; Pred HR: {pred_hr}')

In [ ]:
data